# ACCESS THREDDS FOR HYCOM

Alli Ho 08/23/2021

Updated 04/05/2022 by Jerry

Using Jerry's code `access_thredds.ipynb` to access HYCOM forecast from https://tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0/FMRC/GLBy0.08_930_FMRC_best.ncd.html

#### Adjusted code to work with HYCOM variables:

UPDATE: this should no longer by needed (in theory), `thredds_utils.py` should handle HYCOM data normally now.

- `hycom_utils.py` was updated from `thredds_utils.py` to work with HYCOM output


In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

import src.utils as utils
from src.thredds_utils import ThreddsCode, slice_dataset, retrieve_dataset

## SETUP

Change the date and filename for when you want to start your deployment

Typically the forecast starts a week before today's date and runs for 300 hours (12.5 days), and so can forecast ~6 days ahead. 

Automatically set daterange to a week ahead:

In [ ]:
# daterange = ("2021-08-24T00", "2021-09-03T00");
daterange = (np.datetime64("now", "h") - np.timedelta64(1, "D"), np.datetime64("now", "h") + np.timedelta64(7, "D"))
currentname = "hycom_mwbproj";

## DOWNLOAD

In [ ]:
dataset_url = "https://tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0/FMRC/GLBy0.08_930_FMRC_best.ncd"

In [ ]:
# num_chunks = 1
# # EXCLUDE THESE VARIABLES:
# dropvars = {'water_temp', 'water_temp_bottom','salinity', 'salinity_bottom','water_u_bottom','water_v_bottom','surf_el'}

# thredds_data = {
#     thredds_utils.DATA_HYCOMFORE: xr.open_dataset(dataset_url, chunks={"time": 1},decode_times=False, drop_variables=dropvars),
# }
# t0 = np.datetime64(thredds_data[thredds_utils.DATA_HYCOMFORE].time.units[12:35])
# tmp = thredds_data[thredds_utils.DATA_HYCOMFORE].time.data
# # thredds_data[thredds_utils.DATA_HYCOMFORE]["timeday"] = t0 + np.array(tmp,dtype="timedelta64[h]")
# thredds_data[thredds_utils.DATA_HYCOMFORE]["t0"] = np.timedelta64(t0 - np.datetime64('0000-01-01T00:00:00.000'), 'h')/np.timedelta64(1, 'D')
# thredds_data[thredds_utils.DATA_HYCOMFORE] = thredds_data[thredds_utils.DATA_HYCOMFORE].assign_coords(time=(t0 + np.array(tmp,dtype="timedelta64[h]")))


# thredds_data[thredds_utils.DATA_HYCOMFORE]["time"].attrs["long_name"] = "Forecast time"
# thredds_data[thredds_utils.DATA_HYCOMFORE]["time"].attrs["standard_name"] = "time"
# thredds_data[thredds_utils.DATA_HYCOMFORE]["time"].attrs["_CoordinateAxisType"] = "Time"
# thredds_data[thredds_utils.DATA_HYCOMFORE]["tau"].attrs["units"] = "hours since " + thredds_data[thredds_utils.DATA_HYCOMFORE]["tau"].time_origin

hycom_data = retrieve_dataset(ThreddsCode.DATA_HYCOMFORE)

In [ ]:
print("END OF FORECAST: "+ str(hycom_data["time"].data[-1]))
hycom_data

In [ ]:
region_data = [
   {
      "name": currentname,
      "code": ThreddsCode.DATA_HYCOMFORE,
      "time_range": daterange,
      "lat_range": (22, 45),
      "lon_range": (273, 295),
      "include_range_endpoints": False
   }
]

In [ ]:
regions = []
for rd in region_data:
    dataset = slice_dataset(
        rd["code"], rd["time_range"], rd["lat_range"],
        rd["lon_range"], inclusive=rd["include_range_endpoints"]
    )
    new_reg = {"name": rd["name"], "dataset": dataset}
    regions.append(new_reg)
    print(f"region {new_reg['name']} data megabytes: {new_reg['dataset'].nbytes / 1024 / 1024}")

In [ ]:
for i, r in enumerate(regions):
    save_dir = utils.create_path(utils.CURRENT_NETCDF_DIR)
    filename = f"{r['name']}.nc"
    # save file
    r["dataset"].to_netcdf(save_dir / filename)
    print(f"saved to {save_dir / filename}")
print("done")